In [1]:
import numpy as np
import pandas as pd
from pandas import json_normalize
import seaborn as sns
import matplotlib.pyplot as plt

import time
import datetime

import json
import os
from dotenv import load_dotenv #pip install python-dotenv

from bs4 import BeautifulSoup
import requests 

from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# comprovar quantes requests he fet i quantes me'n queden

url_search = "https://api.discogs.com/database/search"

params = {
    "type": "release"  # Search for masters
    , "format": "album"           # just search for albums
    , "per_page": 100    # Number of results per page (max 100)
    , "page": 1          # Start with page 1
    , 'country': 'UK'
    , "year": 2015
    , 'genre': 'Rock'
    # , "style": style
}

headers = {"User-Agent": "MyDiscogsApp/1.0", "Authorization": "Discogs token=UwfqmsztxwnfABgQpmhaAsprbUgpOJKGOJSQAqfp"}
response = requests.get(url_search, headers=headers, params=params)

rate_limit = response.headers.get("X-Discogs-Ratelimit")
rate_limit_used = response.headers.get("X-Discogs-Ratelimit-Used")
rate_limit_remaining = response.headers.get("X-Discogs-Ratelimit-Remaining")

print(f"Rate Limit: {rate_limit}")
print(f"Rate Limit Used: {rate_limit_used}")
print(f"Rate Limit Remaining: {rate_limit_remaining}")

Rate Limit: 60
Rate Limit Used: 0
Rate Limit Remaining: 60


### **Proves**: un any en un país

In [2]:
# get all rock_styles in one year in one country

url = "https://api.discogs.com/database/search"
headers = {"User-Agent": 'Arnau', "Authorization": "Discogs token=UwfqmsztxwnfABgQpmhaAsprbUgpOJKGOJSQAqfp"}

genres = ["Blues", "Classical", "Electronic", "Folk, World, & Country", "Funk / Soul", "Hip-Hop", "Jazz", "Latin", "Pop", "Rock", "Stage & Screen"]
rock_styles = ["Pop Rock", "Punk", "Alternative Rock", "Indie Rock", "Hardcore", "Rock & Roll", "Hard Rock", "Psychedelic Rock"
              , "Black Metal", "Heavy Metal", "Folk Rock", "Experimental", "Prog Rock", "Death Metal", "Garage Rock"]

for style in rock_styles:
    # Define parameters for artist search
    params = {
        "per_page": 100    # Number of results per page (max 100)
        , 'type': 'release'
        , 'format': 'album'
        , 'genre': 'Rock'

        # items to change
        , 'country': 'UK'
        , 'year': 2015
        , 'style': style
    }

    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    items_number = data['pagination']['items']  # number of items

    print(f"{style}: {items_number}")

Pop Rock: 453
Punk: 904
Alternative Rock: 731
Indie Rock: 976
Hardcore: 254
Rock & Roll: 154
Hard Rock: 312
Psychedelic Rock: 633
Black Metal: 146
Heavy Metal: 197
Folk Rock: 326
Experimental: 465
Prog Rock: 399
Death Metal: 133
Garage Rock: 141


In [3]:
# get one style in one year in one country

url = "https://api.discogs.com/database/search"
headers = {"User-Agent": "MyDiscogsApp/1.0", "Authorization": "Discogs token=UwfqmsztxwnfABgQpmhaAsprbUgpOJKGOJSQAqfp"}

# Define parameters for artist search
params = {
    "per_page": 100    # Number of results per page (max 100)
    , 'type': 'release'
    , 'format': 'album'
    , 'page': 1

    # items to change
    , 'country': 'UK'
    , 'year': 2015
    , 'genre': 'Rock'
    # , 'style': 'Alternative Rock'
}

response = requests.get(url, headers=headers, params=params)
data = response.json()
items_number = response.json()['pagination']['items']  # number of items

print(f"Items: {items_number}")

# response.json()

Items: 6360


### **Rock masters from a specific country**

In [4]:
df_masters_scrapped = pd.read_csv('Datasets/df_masters.csv')
df_masters_scrapped.shape

(14058, 13)

In [25]:
url_search = "https://api.discogs.com/database/search"
headers = {"User-Agent": "MyDiscogsApp", "Authorization": "Discogs token=UwfqmsztxwnfABgQpmhaAsprbUgpOJKGOJSQAqfp"}

df_masters_scrapped = pd.read_csv('Datasets/df_masters.csv')

# rock_styles = ["Pop Rock", "Punk", "Alternative Rock", "Indie Rock", "Hardcore", "Rock & Roll", "Hard Rock", "Psychedelic Rock"
#               , "Black Metal", "Heavy Metal", "Folk Rock", "Experimental", "Prog Rock", "Death Metal", "Garage Rock"]

rock_styles = ["Prog Rock", "Death Metal", "Garage Rock"]

# create empty lists
artist_id_list = []
artists_list = []
titles_list = []
years_list = []
release_countries_list = []
album_length_list = []
tracks_list = []
release_type_list = []
genres_list = []
styles_list = []
master_id_new_list = []
artist_profile_list = []
main_release_id_list = []

attempts = 0
scrapped = 0
start_year = 2015

for year in range(start_year, start_year-1, -1):
    print(f'Year: {year}')

    for style in rock_styles:
        print(f'Style: {style}')

        # Define parameters for album search
        params = {
            "type": "release"  # Search for masters
            , "format": "album"           # just search for albums
            , "per_page": 100    # Number of results per page (max 100)
            , "page": 1          # Start with page 1
            , 'country': 'UK'
            , "year": year
            , 'genre': 'Rock'
            , "style": style
        }

        response1 = requests.get(url_search, headers=headers, params=params)
        time.sleep(1)
        data1 = response1.json()

        items = data1['pagination']['items']
        print(f'Items: {items} releases')

        pages = data1['pagination']['pages']+1
        
        for page in range(1, pages):
            print(f'------------ Page: {page} of {pages-1}, {style} of {year} -------------')

            params['page'] = page     # change pages
            response2 = requests.get(url_search, headers=headers, params=params)
            data2 = response2.json()

            for id in range(len(data2['results'])):
                attempts+=1
                time.sleep(1)

                master_id = data2['results'][id]['master_id']

                if master_id == 0:
                    print(f'{attempts} attempts - {scrapped} scrapped - {((page-1)*100)+id+1}/{items} - master_id {master_id}')
                    continue        # I cannot get info 
                elif master_id in master_id_new_list:
                    print(f'{attempts} attempts - {scrapped} scrapped - {((page-1)*100)+id+1}/{items} - master_id {master_id} already in list') 
                    continue        # there are several releases of the same master_release, I just want one (the master_release)
                elif master_id in df_masters_scrapped['master_id'].values:
                    print(f'{attempts} attempts - {scrapped} scrapped - {((page-1)*100)+id+1}/{items} - master_id {master_id} already scrapped') 
                    continue        # I have already scrapped it before
                else:
                    url_master = f"https://api.discogs.com/masters/{master_id}"
                    response3 = requests.get(url_master, headers=headers)
                    time.sleep(1)
                    data3 = response3.json()

                    artist = data3['artists'][0]['name']

                    if artist == 'Various':
                            print(f'{attempts} attempts - {scrapped} scrapped - {((page-1)*100)+id+1}/{items} - master_id {master_id}: Compilation') 
                            continue   # I don't want compilations
                    else:
                    # define the data I want to get
                        title = data3['title']
                        release_year = data3['year']                 
                        genres = data3['genres']
                        styles = data3['styles']

                    # a more complex information is the total duration of the album, I have to sum the durations of each song
                        number_of_songs = len(data3['tracklist'])
                        song_durations = []

                        for song in range(number_of_songs):
                            song_duration = data3['tracklist'][song]['duration']
                            try:
                                minutes = int(song_duration.split(':')[0])
                                seconds = int(song_duration.split(':')[1])
                                song_duration_minutes = minutes + seconds/60
                                song_durations.append(song_duration_minutes)
                            except:
                                pass
                            album_length = round(sum(i for i in song_durations), 2)

                        main_release_id = data3['main_release']
                        artist_id = data3['artists'][0]['id']

                    # release page
                        url = f"https://api.discogs.com/releases/{main_release_id}"
                        time.sleep(1)
                        response4 = requests.get(url, headers=headers)
                        data4 = response4.json()

                    # define the data I want to get
                        release_country = data4['country']
                        release_type = data4['formats'][0]['descriptions']

                    # artist page
                        url = f"https://api.discogs.com/artists/{artist_id}"
                        time.sleep(1)
                        response5 = requests.get(url, headers=headers)
                        data5 = response5.json()

                    # define the data I want to get
                        try:
                            artist_profile = data5['profile']
                        except:
                            artist_profile = np.nan
                            pass

                    # save the info in the lists
                        artist_id_list.append(str(artist_id))
                        artists_list.append(artist)
                        titles_list.append(title)
                        years_list.append(release_year)
                        genres_list.append(genres)
                        styles_list.append(styles)
                        master_id_new_list.append(master_id)
                        album_length_list.append(album_length)
                        tracks_list.append(number_of_songs)
                        release_countries_list.append(release_country)
                        release_type_list.append(release_type)
                        main_release_id_list.append(main_release_id)

                        if artist_profile == '':
                            artist_profile_list.append(np.nan)
                        else:
                            artist_profile_list.append(artist_profile)

                        scrapped+=1

                    # list lengths
                        lists = [artist_id_list, main_release_id_list, master_id_new_list, release_countries_list, artists_list, titles_list, album_length_list
                                 , tracks_list, release_type_list, genres_list, styles_list, artist_profile_list, years_list]

                    # Check if all lists have the same length
                        lengths = [len(lst) for lst in lists]
                        if len(set(lengths)) != 1:
                            print("Lengths are not the same.")
                            break # stop the loop, I won't be able to store the data if I have one value missing

                        print(f'{attempts} attempts - {scrapped} scrapped - {((page-1)*100)+id+1}/{items} - master_id {master_id}: {artist} - {title}')
                        time.sleep(1)

Year: 2015
Style: Prog Rock
Items: 399 releases
------------ Page: 1 of 4, Prog Rock of 2015 -------------


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Se ha forzado la interrupción de una conexión existente por el host remoto', None, 10054, None))

In [ ]:
# UK 2015

# Scrapped:
# Pop Rock: 453
# Punk: 904
# Alternative Rock: 731
# Indie Rock: 976
# Hardcore: 254
# Rock & Roll: 154
# Hard Rock: 312
# Psychedelic Rock: 633
# Black Metal: 146
# Heavy Metal: 197
# Folk Rock: 326
# Experimental: 465

# To scrap:
# Prog Rock: 399
# Death Metal: 133
# Garage Rock: 141

453+904+731+976+254+154+312+633+146+197+326+465+399+133+141

2922

In [52]:
(112*60)/2922

2.299794661190965

### **Saving the data**

In [7]:
df_masters = pd.DataFrame({'artist_id': artist_id_list
                        , 'master_id': master_id_new_list
                        , 'main_release_id': main_release_id_list
                        , 'release_country': release_countries_list
                        , 'artist': artists_list
                        , 'title': titles_list
                        , 'year': years_list
                        , 'album_length': album_length_list
                        , 'tracks': tracks_list
                        , 'release_type': release_type_list
                        , 'genres': genres_list
                        , 'styles': styles_list
                        , 'artist_profile': artist_profile_list
                        })
df_masters.shape

(1037, 13)

In [8]:
print(f'{df_masters.shape[0]} releases\n{df_masters.artist.nunique()} different artists')

1037 releases
940 different artists


In [9]:
df_masters.sample(10)

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
323,131370,284291,1870107,UK,UK Subs,Mad Cow Fever,1991,0.00,14,"[LP, Album]",[Rock],"[Blues Rock, Punk]",The UK Subs were another group that emerged fr...
183,1247638,1744051,9032472,UK,Billy Liar,Lies Lied Live,2015,31.38,9,[Album],[Rock],[Punk],Acoustic folk-punk singer/songwriter from Edin...
76,10388,931912,1698042,UK,Madness,On Stage - London,2006,102.32,26,"[Album, Limited Edition]",[Rock],[Ska],"Madness are an English ska, 2 tone & pop band ..."
912,65697,848847,1522811,UK,Skullflower,Circulus Vitiosus Deus,2008,190.88,32,[Album],[Rock],"[Abstract, Noise, Experimental, Drone]",Skullflower formed in London in 1987 as an imp...
924,33801,22234,3962039,UK,Current 93,Crooked Crosses For The Nodding God,1989,46.42,17,[Album],"[Electronic, Rock]","[Experimental, Neofolk]",Current 93 is a group led by [a95998] with an ...
405,1266836,799620,6387493,Japan,Hogwash,Rainmaker,2014,0.00,8,[Album],[Rock],"[Pop Punk, Melodic Hardcore]",French melodic punk rock band from Villeneuve ...
844,1763597,999490,8505961,UK,Steve Dowling,Austerity,2015,0.00,9,"[MP3, Album]","[Rock, Funk / Soul, Blues, Pop, Folk, World, &...",[Folk Rock],Galloway based singer songwriter
34,73811,265552,2022139,US,Melanie (2),As I See It Now,1974,39.20,12,"[LP, Album]",[Rock],"[Folk Rock, Pop Rock]","American singer/songwriter, born February 3, 1..."
733,5067590,1142145,8565746,Australia,Gabriella Cohen,Full Closure And No Details,2015,40.68,10,"[LP, Album, Stereo]",[Rock],"[Lo-Fi, Indie Rock]","Brisbane-born, Melbourne-based singer-songwrit..."
354,1689901,837697,7026120,UK,Wetdog,Divine Times,2015,40.67,14,"[LP, Album]",[Rock],"[Post-Punk, Art Rock]",NaN


In [10]:
df_masters.release_country.value_counts().head()

release_country
UK                 590
US                 163
Europe              75
UK & Europe         68
UK, Europe & US     20
Name: count, dtype: int64

In [11]:
df_masters.year.value_counts().head()

year
2015    681
2014     33
2013     18
1982     16
2005     16
Name: count, dtype: int64

### **Concat with the previous df**

In [12]:
# import the last df
df_masters_scrapped = pd.read_csv('Datasets/df_masters.csv')
df_masters_scrapped.shape

(14058, 13)

In [13]:
# concat with the df of the albums I just scrapped
df_masters_concat = pd.concat([df_masters_scrapped, df_masters])
df_masters_concat.reset_index(drop=True, inplace=True)
df_masters_concat.shape

(15095, 13)

In [14]:
def check_duplicates(df):
    if df['master_id'].duplicated().sum() == 0:
        print('No duplicates')
    else: 
        duplicates = df['master_id'].duplicated().sum()
        df.drop_duplicates(subset='master_id', inplace=True)
        df.reset_index(drop=True, inplace=True)
        print(f'Found {duplicates} duplicates')
        print(f'Resulting dataset: {df.shape}')

In [15]:
check_duplicates(df_masters_concat)

No duplicates


In [16]:
print(f'{df_masters_concat.shape[0]} releases\n{df_masters_concat.artist.nunique()} different artists')

15095 releases
9146 different artists


### **Exporting to csv**

In [17]:
df_masters_concat.to_csv('Datasets/df_masters.csv', index=False)

In [ ]:
# # import the last df
# df_masters_concat = pd.read_csv('Datasets/df_masters.csv')

### **Statistics**

In [18]:
df_masters_concat.year.value_counts().head(10)

year
2020    1255
2016    1210
2019    1208
2015    1151
2021    1082
2022    1039
2023     975
2018     800
2024     778
2014     641
Name: count, dtype: int64

In [19]:
df_masters_concat.release_country.value_counts().head(10)

release_country
UK                 8357
US                 2164
Europe              949
UK & Europe         739
UK, Europe & US     300
Germany             293
UK & US             247
Worldwide           199
Australia           160
Japan               137
Name: count, dtype: int64

In [22]:
df_masters_concat[df_masters_concat['artist']=='King Gizzard And The Lizard Wizard'].sort_values('year', ascending=False)

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
2673,2532180,3459911,30468428,US,King Gizzard And The Lizard Wizard,Nonagon Infinity Live,2024,41.17,8,"['LP', 'Compilation', 'Limited Edition']",['Rock'],"['Garage Rock', 'Psychedelic Rock']","Psychedelic rock band from Melbourne, Victoria..."
10717,2532180,2079445,18303538,France,King Gizzard And The Lizard Wizard,Demos Vol. 1. (Music To Kill Bad People To),2021,44.47,14,"['LP', 'Album', 'Limited Edition']",['Rock'],['Psychedelic Rock'],"Psychedelic rock band from Melbourne, Victoria..."
10836,2532180,2331730,20525803,UK & Europe,King Gizzard And The Lizard Wizard,Bootleg Box Set,2021,0.00,135,"['Compilation', 'Limited Edition', 'Numbered']",['Rock'],"['Psychedelic Rock', 'Thrash', 'Heavy Metal']","Psychedelic rock band from Melbourne, Victoria..."
10799,2532180,2464735,17317687,UK,King Gizzard And The Lizard Wizard,Chilled Gizzard,2021,0.00,15,"['Album', 'Compilation', 'Stereo']",['Rock'],"['Psychedelic Rock', 'Garage Rock', 'Blues Rock']","Psychedelic rock band from Melbourne, Victoria..."
7577,2532180,2149990,18939442,Australia,King Gizzard And The Lizard Wizard,Live In Sydney '21,2021,89.62,16,['FLAC'],['Rock'],['Psychedelic Rock'],"Psychedelic rock band from Melbourne, Victoria..."
10728,2532180,2075629,18075865,France,King Gizzard And The Lizard Wizard,Demos Vol. 2. (Music To Eat Bananas To),2021,47.12,14,"['LP', 'Album', 'Limited Edition']",['Rock'],['Psychedelic Rock'],"Psychedelic rock band from Melbourne, Victoria..."
7667,2532180,1996411,17581906,Australia,King Gizzard And The Lizard Wizard,L​.​W.,2021,42.10,9,"['FLAC', 'Album']",['Rock'],['Psychedelic Rock'],"Psychedelic rock band from Melbourne, Victoria..."
10723,2532180,2037586,17939092,Australia,King Gizzard And The Lizard Wizard,Live In Melbourne '21,2021,88.85,18,"['FLAC', 'Album']",['Rock'],['Psychedelic Rock'],"Psychedelic rock band from Melbourne, Victoria..."
8727,2532180,1897413,14661823,Australia,King Gizzard And The Lizard Wizard,Live In Brussels '19,2020,95.42,17,"['FLAC', 'Album']",['Rock'],['Psychedelic Rock'],"Psychedelic rock band from Melbourne, Victoria..."
10807,2532180,1888098,16620795,Australia,King Gizzard And The Lizard Wizard,Teenage Gizzard,2020,29.45,10,"['FLAC', 'Compilation']",['Rock'],"['Psychedelic Rock', 'Surf']","Psychedelic rock band from Melbourne, Victoria..."


In [23]:
df_masters_concat.artist.value_counts().head(10)

artist
The Fall                              44
Bob Dylan                             28
King Gizzard And The Lizard Wizard    24
UK Subs                               22
This Is The Bridge                    21
Six By Seven                          21
The Levellers                         20
King Crimson                          18
Schroedinger's Cat                    18
Marillion                             18
Name: count, dtype: int64

In [24]:
df_masters_concat.genres.value_counts().head(10)

genres
['Rock']                              10004
['Electronic', 'Rock']                 1044
['Rock', 'Pop']                         983
[Rock]                                  655
['Rock', 'Folk, World, & Country']      479
['Electronic', 'Rock', 'Pop']           229
['Jazz', 'Rock']                        123
['Rock', 'Blues']                       104
[Rock, Pop]                             100
[Electronic, Rock]                       97
Name: count, dtype: int64

In [107]:
df_masters_concat.styles.value_counts().head(10)

styles
['Indie Rock']                        770
['Punk']                              491
['Alternative Rock']                  437
['Black Metal']                       391
[]                                    381
['Prog Rock']                         378
['Psychedelic Rock']                  341
['Death Metal']                       242
['Hard Rock']                         226
['Alternative Rock', 'Indie Rock']    219
Name: count, dtype: int64

In [78]:
df_masters_concat[df_masters_concat['release_country']=='Europe'].sort_values('artist')

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
1829,156155,118883,737915,Europe,3 Mustaphas 3,Shopping,1987,44.72,9,"['LP', 'Album']","['Rock', 'Folk, World, & Country']",['Folk Rock'],British band [b]3 Mustaphas 3[/b] did much to ...
4689,3750880,1706746,15028328,Europe,5 Seconds Of Summer,CALM,2020,39.50,12,['Album'],"['Rock', 'Pop']","['Pop Rock', 'Power Pop']",5 Seconds Of Summer (5SOS) is an Australian po...
1326,103279,219113,2075252,Europe,60ft Dolls,The Big 3,1996,41.67,13,['Album'],['Rock'],"['Britpop', 'Indie Rock', 'Hard Rock']","Rock trio from Newport, South Wales, in the st..."
2899,14085388,3566297,31387334,Europe,86TVs,86TVs,2024,0.00,15,"['LP', 'Album', 'Limited Edition']","['Rock', 'Pop']","['Alternative Rock', 'Indie Rock']","Made up of [a499923] Felix and Hugo White, alo..."
10763,8375823,2063863,17972098,Europe,A/lpaca,Make It Better,2021,0.00,9,"['LP', 'Album', 'Limited Edition', 'Stereo']",['Rock'],"['Psychedelic Rock', 'Space Rock', 'Garage Rock']","Mantua, Italy\r\n"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3780,6774153,3038081,26603456,Europe,boygenius,The Record,2023,42.32,12,['Album'],['Rock'],"['Folk Rock', 'Indie Rock']","Phoebe Bridgers, Lucy Dacus and Julien Baker f..."
1088,3971017,725754,6023794,Europe,frnkiero andthe cellabration.,.Stomachaches.,2014,0.00,12,['Album'],['Rock'],[],NaN
3744,12268465,3149436,27982503,Europe,sign crushes motorist,Hurting,2023,30.42,10,"['Album', 'Limited Edition', 'Stereo']",['Rock'],"['Alternative Rock', 'Indie Rock', 'Noisecore']",Add as Sign Crushes Motorist
9400,5609390,1558662,13757895,Europe,whenyoung,Reasons To Dream,2019,41.67,11,['Album'],['Rock'],['Alternative Rock'],"Whenyoung is an Irish indie rock group, origin..."


In [79]:
percentage_album_length_missing = round((df_masters_concat['album_length']==0).sum() / df_masters_concat.shape[0] * 100, 2)
print(f'% missing album_length: {percentage_album_length_missing}%')

% missing album_length: 41.97%


In [80]:
df_masters_concat['artist_profile'].isna().sum()

1838

In [81]:
percentage_artist_profile_missing = round(df_masters_concat['artist_profile'].isna().sum() / df_masters_concat.shape[0] * 100, 2)
print(f'% missing artist_profile: {percentage_artist_profile_missing}%')

% missing artist_profile: 14.02%


In [82]:
df_masters_concat[df_masters_concat['artist_profile']==''] # should be empty

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile


In [19]:
df_masters_concat.loc[10, 'artist_profile']

'English Pop punk band from Southend-on-Sea, Essex, consisting of James Bourne, Matt Willis and Charlie Simpson. Formed in 2000, the band had four UK number-one singles, won two Brit awards and have released four studio albums, selling in excess of five million records.\r\n\r\nIn 2003, Dutch releases of this band were released under the artist name "Bust" to avoid confusion with Dutch band [a493201] who had a hit back then with "We Want More".'

In [84]:
df_masters_concat.groupby(["release_country", 'year']).agg({'master_id': 'count'}).sort_values('master_id', ascending=False).astype(int).head(15)

master_id
release_country year           
UK              2020        741
                2019        717
                2016        669
                2021        602
                2022        577
                2023        551
                2024        428
                2014        308
                2015        267
US              2016        173
                2020        132
                2019        125
                2022        116
                2014        105
                2021         96